# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [27]:
import ollama
from IPython.display import Markdown, display, update_display

MODEL_LLAMA = 'llama3'  # Asegúrate de que este modelo esté disponible en tu instancia de Ollama

# Configura el cliente nativo de Ollama
ollama_native = ollama.Client(
    host='http://localhost:11434'
)

question = """
Explica qué hace este código y por qué:
yield from (book.get("author") for book in books if book.get("author"))
"""
system_prompt = "Eres un tutor técnico útil que responde preguntas sobre código Python, ingeniería de software, ciencia de datos y LLM"
user_prompt = f"Por favor, da una explicación detallada de la siguiente pregunta: {question}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

print(f"**Respuesta de {MODEL_LLAMA} (usando el cliente nativo de Ollama):**")
response_ollama_native = ollama_native.chat(
    model=MODEL_LLAMA,
    messages=messages,
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in response_ollama_native:
    response += chunk['message']['content'] or ''
    response = response.replace("```", "").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

**Respuesta de llama3 (usando el cliente nativo de Ollama):**


Vamos a analizar el código step by step.

El código que proporcionas es un ejemplo de uso de recursion y yield en Python. Vamos a explicar qué hace cada parte del código.

**Nota:** Es importante mencionar que la variable `books` no está definida en este snippet, por lo que asumiremos que se refiere a una lista o algún objeto que contiene información sobre libros (título, autor, etc.).

**Parágrafos**

La función principal es `(book.get("author") for book in books if book.get("author"))`.

Esta expresión se conoce como un `generator expression`. Es una forma compacta de crear un generator.

Un generator es una función que devuelve un iterador, en lugar de una lista o otro tipo de colección. Los generators permiten ahorrar memoria y recursos computacionales al no almacenar toda la colección en memoria a la vez.

**`book.get("author")`**

Esta expresión intenta obtener el valor de la clave `"author"` desde el diccionario `book`. Si la clave no existe, devuelve `None`.

**`for book in books if book.get("author")`**

Este es un filtro para seleccionar solo aquellos diccionarios `book` que tienen una clave `"author"` presente. De esta manera, se evita iterar sobre diccionarios vacíos o que carecen de información.

**`yield from`**

El término `yield from` fue introducido en Python 3.3. Se utiliza para "pasar el control" a otro generator. En otras palabras, crea un nuevo generator y lo convierte en su propio sub-generator.

En este caso específico, se utiliza `yield from` para "pasar el control" al generator interno `(book.get("author") for book in books if book.get("author"))`.

**El comportamiento del código**

Ahora que hemos explicado cada parte del código, veamos cómo se comporta.

1. Se crea un generator que intenta obtener el valor de la clave `"author"` desde cada diccionario `book` en la lista `books`.
2. El filtro `(for book in books if book.get("author"))` selecciona solo aquellos diccionarios que tienen información sobre autores.
3. `yield from` pasa el control al generator interno, permitiendo que se ejecute por sí solo.
4. Cada iteración del generator intenta obtener el valor de la clave `"author"` y lo devuelve (o `None`, si no existe).
5. La función devuelve un iterator iterable sobre estos valores obtenidos.

**Por qué este código es útil**

Este código puede ser utilizado en varias situaciones:

*   Obtener una lista de autores de libros desde una base de datos que tiene información sobre libros y autores.
*   Filtrar y procesar información de manera eficiente, sin almacenar toda la información en memoria a la vez.

Recuerda que este código se utiliza para obtener un conjunto de valores a partir de otro conjunto, por lo que debe ser tratado con precaución si no se comprenderán las condiciones de la data.

In [30]:
import os
from dotenv import load_dotenv
import ollama
import openai
from IPython.display import Markdown, display
import threading

# Cargar variables de entorno desde .env
load_dotenv()

MODEL_LLAMA = 'llama3'
OLLAMA_HOST = 'http://localhost:11434'
MODEL_OPENAI = "gpt-4o-mini"  # Puedes ajustar el modelo de OpenAI si lo deseas

# Obtener la API key de OpenAI desde las variables de entorno
OPENAI_API_KEY ="***tjO8Z6AFbBeTLgRT***"

def obtener_respuesta_ollama(question, system_prompt, model_name=MODEL_LLAMA, host=OLLAMA_HOST):
    try:
        ollama_native = ollama.Client(host=host)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        response_ollama_native = ollama_native.chat(
            model=model_name,
            messages=messages,
            stream=True
        )
        response = ""
        for chunk in response_ollama_native:
            response += chunk['message']['content'] or ''
        return response
    except ollama.exceptions.OllamaError as e:
        return f"Error al interactuar con Ollama: {e}"
    except Exception as e:
        return f"Ocurrió un error inesperado con Ollama: {e}"

def obtener_respuesta_openai(question, system_prompt, model_name=MODEL_OPENAI):
    if not OPENAI_API_KEY:
        return "Error: La API key de OpenAI no está configurada en el archivo .env"
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        response_stream = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
            stream=True,
        )
        response = ""
        for chunk in response_stream:
            if chunk.choices:
                response += chunk.choices[0].delta.content or ""
        return response
    except openai.APIError as e:
        return f"Error de la API de OpenAI: {e}"
    except Exception as e:
        return f"Ocurrió un error inesperado con OpenAI: {e}"

question = """
Explica qué hace este código y por qué:
yield from (book.get("author") for book in books if book.get("author"))
"""
system_prompt = "Eres un tutor técnico útil que responde preguntas sobre código Python, ingeniería de software, ciencia de datos y LLM"

def run_ollama():
    print(f"**Respuesta de {MODEL_LLAMA} (usando el cliente nativo de Ollama):**")
    global respuesta_ollama
    respuesta_ollama = obtener_respuesta_ollama(question, system_prompt)

def run_openai():
    print(f"**Respuesta de {MODEL_OPENAI} (usando la API de OpenAI):**")
    global respuesta_openai
    respuesta_openai = obtener_respuesta_openai(question, system_prompt)

# Crear y ejecutar los hilos en paralelo
ollama_thread = threading.Thread(target=run_ollama)
openai_thread = threading.Thread(target=run_openai)

ollama_thread.start()
openai_thread.start()

ollama_thread.join()
openai_thread.join()

# Mostrar las respuestas después de que ambos hilos hayan terminado
display(Markdown(respuesta_ollama))
display(Markdown(respuesta_openai))

**Respuesta de llama3 (usando el cliente nativo de Ollama):**
**Respuesta de gpt-4o-mini (usando la API de OpenAI):**


Este código utiliza la sintaxis `yield from` en Python para generar una secuencia de autoras de libros (`authors`) a partir de una lista de diccionarios (`books`). Aquí te explico qué hace cada parte:

*   `(book.get("author") for book in books if book.get("author"))`: 
    Este es un **generador** que se crea por la sintaxis `for` y tiene la forma `(expression for variable in iterable if condition)`. Generadores son programas que generan secuencias de resultados a medida que se los pide.

    En este caso, se intenta acceder a cada libro (`book`) en la lista (`books`) y si la clave ("author") existe (`if book.get("author"`), se devuelve el valor asociado. Se utiliza `.get()` para evitar un error de `KeyError` si no existe una clave en el diccionario.

    Por lo tanto, esta secuencia genera un iterable que recorre los libros que contienen la clave ("author") y retorna el autor correspondiente.

*   `yield from(...)`: Es un operador `yield from`, usado para generar una secuencia de resultados de otro generador. Cualquier llamada a yield `from` genera una secuencia de los valores por yield del generador pasado.

En resumen, se usa este código en un contexto de procesamiento de datos, quizá para extraer un conjunto de valores de un conjunto de diccionarios (`books`). 

Por ejemplo:

```python
import json

# suponiendo que books es una lista de diccionario json
books = [
    {"title": "Libro1", "author": "AuthorX"},
    {"title": "Libro2", "author": "AuthorY"},
    # ...
]

# obtener las autoras
def get_authors():
    yield from (book.get("author") for book in books if book.get("author"))

# utilizar el generador
for author in get_authors():
    print(author)
```

Este código se ejecutaría de la siguiente manera:

1.  Se define una lista `books` donde cada elemento es un diccionario que representa un libro con su título y autor.
2.  Se define un generador `get_authors()` que utiliza `(book.get("author") for book in books if book.get("author"))` para obtener solo las autoras de los libros que tienen una clave ("author") presente en sus diccionarios.
3.  El uso de `yield from` significa que en lugar de devolver los valores directamente, se llama al generador interno `(book.get("author") for book in books if book.get("author"))`. Este generador es responsable de generar y devolver la secuencia de autoras según el filtro.
4.  Finalmente, en un bucle `for`, se itera sobre los valores del generador obteniendo solo las autoras.

Error de la API de OpenAI: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_issuer'}}